In [76]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
options(repr.matrix.max.rows = 30)

set.seed(8205)

ERROR: Error in library(themis): there is no package called ‘themis’


 # Reading in Heart Disease Data

In [ ]:

#download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data",
            #  destfile = "./switzerland_data.data")

#read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/switzerland.data/processed.switzerland.data")

cleveland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", col_names = FALSE) |>
    rename(age      = X1,      # Age in years
           sex      = X2,      # Sex (1 = male, 0 = female)
           cp       = X3,      # Chest pain type 
                               # - 1 : typical angina
                               # - 2 : atypical angina
                               # - 3 : non-anginal pain
                               # - 4 : asymptomatic
           trestbps = X4,      # Resting blood pressure (mmHg)
           chol     = X5,      # Serum cholestoral (mg/dL)
           fbs      = X6,      # Fasting blood sugar > 120 mg/dL (1 = true, 0 = false)
           restecg  = X7,      # Resting electrocardiographic results 
                               # - 0 : normal,
                               # - 1 : having ST-T wave abnormality (T wave inversions and/or ST 
                               #       elevation or depression of > 0.05 mV)
                               # - 2 : showing probable or definite left ventricular hypertrophy by Estes' criteria
           thalach  = X8,      # Maximum heart rate achieved
           exang    = X9,      # Exercise induced angina (0 = no, 1 = yes)
           oldpeak  = X10,     # ST depression induced by exercise relative to rest
           slope    = X11,     # The slope of the peak exercise ST segment
                               # - 1 : upsloping
                               # - 2 : flat
                               # - 3 : downsloping
           ca       = X12,     # Number of major vessels (0-3) colored by flourosopy
           thal     = X13,     # 3 = normal; 6 = fixed defect; 7 = reversable defect
           num      = X14)     # diagnosis of heart disease (angiographic disease status)

cleveland_data

# Data wrangling

In [ ]:
cleveland_data <- cleveland_data |>
    mutate(age      = as.integer(age),
           sex      = as.factor(sex),
           cp       = as.factor(cp),
           trestbps = as.integer(trestbps),
           chol     = as.integer(chol),
           fbs      = as.factor(fbs),
           restecg  = as.factor(restecg),
           thalach  = as.integer(thalach),
           exang    = as.factor(exang),
           oldpeak  = as.double(oldpeak),
           slope    = as.factor(slope),
           ca       = as.integer(ca),
           thal     = as.factor(thal),
           num      = as.factor(num))

cleveland_data

In [ ]:
# Splitting the data into training and testing sets
cleveland_split <- initial_split(cleveland_data, prop = 0.75, strata = num)
cleveland_training <- training(cleveland_split) 
cleveland_testing <- testing(cleveland_split)

# Exploratory data analysis

### Heart Disease Summary

In [ ]:
target_summary <- cleveland_training |>
    group_by(num) |>
    summarize(count = n())
target_summary

In [ ]:
target_summary_plot <- target_summary |>
    ggplot(aes(x =  num, y = count, fill = num)) +
    geom_bar(stat = "identity") +
    labs(x = "Type", y = "Count", fill = "Type of Heart Disease") +
    theme(text = element_text(size = 15))

target_summary_plot

From the dataset, the majority of patients do not have any presence of heart disease. For those who have presence of a heart disease, most have **Type 1** whereas **Type 4** is the least common.

### Demographics

In [ ]:
demographic_summary <- cleveland_training |>
    group_by(sex) |>
    summarize(count = n(), average_age = mean(age))
demographic_summary

In [ ]:
demographic_sex_summary_plot <- demographic_summary |>
    ggplot(aes(x =  sex, y = count, fill = sex)) +
    geom_bar(stat = "identity") +
    labs(x = "Sex", y = "Count", fill = "Sex") +
    scale_fill_discrete(labels = c("Female", "Male")) +
    theme(text = element_text(size = 15))

demographic_age_summary_plot <- cleveland_training |>
    ggplot(aes(x =  age, fill = sex)) +
    geom_histogram(binwidth = 2) +
    labs(x = "Age", y =  "Count", fill = "Sex") +
    scale_fill_discrete(labels = c("Female", "Male")) +
    facet_grid(rows = vars(sex)) +
    theme(text = element_text(size = 15))

demographic_sex_summary_plot
demographic_age_summary_plot

The number of male double that of female participants. The average age for this dataset is around 54 years old. In addition, the average age of the male is slightly younger than female participants.

### Blood Pressure

In [ ]:
average_restbps <- cleveland_training |>
    summarize(average_restbps = mean(trestbps))
average_restbps

The average resting blood pressure of all participants is 132*mmHg*. Let's see how this would affect the type of heart disease.

In [ ]:
average_restbps_by_type <- cleveland_training |>
    group_by(num) |>
    summarize(average_restbps = mean(trestbps))
average_restbps_by_type

In [ ]:
restbps_summary_plot <- cleveland_training |>
    ggplot(aes(x = trestbps, fill = num)) +
    geom_histogram(binwidth = 5) +
    geom_vline(xintercept = 132.33, linetype = "dashed", size = 1.0) +
    labs(x = "Resting Blood Pressure (mmHg)", fill = "Type of Heart Disease") +
    facet_grid(rows = vars(num)) +
    theme(text = element_text(size = 15))

restbps_summary_plot

People with no presence of heart disease have lower average resting blood pressure. Those who do tend to have slightly higher resting blood pressure with worse cases of heart disease.

### Serum Cholestoral

In [ ]:
average_chol <- cleveland_training |>
    summarize(average_chol = mean(chol))
average_chol

The average serum cholestoral in the data set is 248.5*mg/dL*. Let's see what's the average for each type of heart disease.

In [ ]:
average_chol_by_type <- cleveland_training |>
    group_by(num) |>
    summarize(average_chol = mean(chol))
average_chol_by_type

In [ ]:
chol_summary_plot <- cleveland_training |>
    ggplot(aes(x = chol, fill = num)) +
    geom_histogram(binwidth = 15) +
    geom_vline(xintercept = 248.5, linetype = "dashed", size = 1.0) +
    labs(x = "Serum Cholestoral (mg/dL)", y = "Count", fill = "Type of Heart Disease") +
    facet_grid(rows = vars(num)) +
    theme(text = element_text(size = 15))

chol_summary_plot

### Fasting Blood Sugar

In [ ]:
#pull the number of instances of each type of heart disease
disease_count <- c(123, 42, 26, 27, 8)

fbs_summary <- cleveland_training |>
    group_by(num, fbs) |>
    summarize(count = n()) 

fbs_lower_than_proportion <- fbs_summary |>
    filter(fbs == 0) |> 
    bind_cols(type_count = disease_count) |>
    mutate(lower_than_proportion = count / type_count) |>
    select(num, lower_than_proportion)

fbs_summary
fbs_lower_than_proportion

In [ ]:
fbs_summary_plot <- cleveland_training |>
    ggplot(aes(x = num, fill = fbs)) +
    geom_bar(position = "fill") +
    labs(x = "Type of Heart Disease", y = "Proportion", fill = "Fasting blood sugar \ngreater than 120mg/dL") +
    scale_fill_discrete(labels = c("False", "True")) +
    theme(text = element_text(size = 15))
fbs_summary_plot

### Resting electrocardiographic

In [ ]:
restecg_summary <- cleveland_training |>
    group_by(num, restecg) |>
    summarize(count = n()) |>
    filter(restecg == 0) |> 
    bind_cols(type_count = disease_count) |>
    mutate(normal_proportion = count / type_count) |>
    select(num, normal_proportion)
restecg_summary

In [ ]:
restecg_summary_plot <- cleveland_training |>
    ggplot(aes(x = num, fill = restecg)) +
    geom_bar(position = "fill") +
    labs(x = "Type of Heart Disease", y = "Proportion", fill = "Resting electrocardiographic results") +
    scale_fill_discrete(labels = c("Normal", "ST-T wave abnormality", "Probable/definite \nleft ventricular hypertrophy")) +
    theme(text = element_text(size = 15))
restecg_summary_plot

### Maximum Heart Rate

In [ ]:
average_thalach <- cleveland_training |>
    summarize(average_thalach = mean(thalach))
average_thalach

In [ ]:
average_thalach_by_type <- cleveland_training |>
    group_by(num) |>
    summarize(average_thalach = mean(thalach))
average_thalach_by_type

In [ ]:
thalach_summary_plot <- cleveland_training |>
    ggplot(aes(x = thalach, fill = num)) +
    geom_histogram(binwidth = 5) +
    geom_vline(xintercept = 149.5, linetype = "dashed", size = 1.0) +
    facet_grid(rows = vars(num)) +
    labs(x = "Maximum Heart Rate Achieved", fill = "Type of Heart Disease")

thalach_summary_plot

### Exercise Induced Angina

In [ ]:
exang_summary <- cleveland_training |>
    group_by(num, exang) |>
    summarize(count = n()) 

exang_proportion <- exang_summary |>
    filter(exang == 0) |> 
    bind_cols(type_count = disease_count) |>
    mutate(exang_proportion = count / type_count) |>
    select(num, exang_proportion)

exang_summary
exang_proportion

In [ ]:
exang_summary_plot <- cleveland_training |>
    ggplot(aes(x = num, fill = exang)) +
    geom_bar(position = "fill") +
    labs(x = "Type of Heart Disease", y = "Proportion", fill = "Excercise induced angina") +
    scale_fill_discrete(labels = c("False", "True")) +
    theme(text = element_text(size = 15))
exang_summary_plot

### ST Depression

In [ ]:
average_oldpeak <- cleveland_training |>
    summarize(average_oldpeak = mean(oldpeak))
average_oldpeak

In [ ]:
average_oldpeak_by_type <- cleveland_training |>
    group_by(num) |>
    summarize(average_oldpeak = mean(oldpeak))
average_oldpeak_by_type

In [ ]:
oldpeak_summary_plot <- cleveland_training |>
    ggplot(aes(x = oldpeak, fill = num)) +
    geom_histogram(binwidth = 0.2) +
    geom_vline(xintercept = 1.033, linetype = "dashed", size = 1.0) +
    facet_grid(rows = vars(num))

oldpeak_summary_plot

### Slope of the Peak Exercise ST Segment

In [ ]:
slope_summary <- cleveland_training |>
    group_by(num, slope) |>
    summarize(count = n()) 
slope_summary

In [ ]:
slope_summary_plot <- cleveland_training |>
    ggplot(aes(x = num, fill = slope)) +
    geom_bar(position = "fill") +
    labs(x = "Type of Heart Disease",y = "Proportion", fill = "Slope of peak exercise ST segments") +
    scale_fill_discrete(labels = c("Upsloping", "Flat", "Downsloping")) +
    theme(text = element_text(size = 15))
slope_summary_plot

### Number of major vessels colored by flourosopy

In [ ]:
ca_summary <- cleveland_training |>
    filter(!is.na(ca)) |>
    group_by(num, ca) |>
    summarize(count = n()) 

ca_summary

In [ ]:
ca_summary_plot <- cleveland_training |>
    filter(!is.na(ca)) |>
    ggplot(aes(x = num, fill = as.factor(ca))) +
    geom_bar(position = "fill") +
    labs(x = "Type of Heart Disease", y = "Proportion",
         fill = "Number of major vessels \ncolored by flourosopy") +
    scale_fill_discrete(labels = c(0, 1, 2, 3)) +
    theme(text = element_text(size = 15))
ca_summary_plot

### Choosing Predictor Variables

In [ ]:
heart_disease_subset <- cleveland_training |>
    select(trestbps,
           chol,
           thalach,
           oldpeak,
           num)
names <- colnames(heart_disease_subset |> select(-num))

In [ ]:
# create an empty tibble to store the results
accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

# create a model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")

# create a 5-fold cross-validation object
heart_disease_vfold <- vfold_cv(heart_disease_subset, v = 5, strata = num)

# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:length(names)) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("num", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        heart_disease_recipe <- recipe(as.formula(model_string), 
                                data = heart_disease_subset) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors()) 

        # tune the KNN classifier with these predictors, 
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(heart_disease_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = heart_disease_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
accuracies